<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
!gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

Downloading...
From: https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph
To: /content/archive.zip
100% 84.9M/84.9M [00:00<00:00, 137MB/s] 
Archive:  archive.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [2]:
!pip install sparsesvd

Импортируем нужные библиотеки:

In [3]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import torch
import nltk
import gensim
import gensim.downloader as api

In [4]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [6]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [6]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [8]:
dev_data[:10]['text']

0               @chrishasboobs AHHH I HOPE YOUR OK!!! 
1    @misstoriblack cool , i have no tweet apps  fo...
2    @TiannaChaos i know  just family drama. its la...
3    School email won't open  and I have geography ...
4                               upper airways problem 
5           Going to miss Pastor's sermon on Faith... 
6              on lunch....dj should come eat with me 
7     @piginthepoke oh why are you feeling like that? 
8      gahh noo!peyton needs to live!this is horrible 
9    @mrstessyman thank you glad you like it! There...
Name: text, dtype: object

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [6]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(" ".join(line))

@ chrishasboobs ahhh i hope your ok !!!


In [7]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

chrishasboobs ahhh hope your


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [8]:
word2vec = api.load("word2vec-google-news-300")

In [ ]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [ ]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [ ]:
from torch.utils.data import Dataset, random_split


class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens

    def get_embeddings_(self, tokens):
        embeddings = [(word2vec.get_vector(w) - self.mean) / self.std for w in tokens if w in word2vec]

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [ ]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [ ]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]
examples = {"features": [np.mean(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [ ]:
examples['features'][:10]

[array([ 7.9884803e-01,  2.9639465e-01, -2.2475016e-01,  8.2761788e-01,
        -3.1564566e-01, -1.6312727e-01,  1.7091973e-01, -2.3886690e-01,
         1.3271519e+00,  7.9067945e-02, -3.7298936e-01, -6.2446344e-01,
        -4.5519450e-01, -6.0943842e-01, -8.8517392e-01,  3.5065675e-01,
         3.0780718e-01,  6.7162788e-01,  4.2289284e-01,  2.1005470e-01,
        -7.7801168e-01,  3.7947506e-01,  6.3729638e-01, -6.4212009e-02,
        -2.1386537e-01,  6.5230477e-01,  5.6758843e-02, -1.4959922e-01,
        -1.0668172e-01,  7.2228706e-01,  3.5502084e-02,  3.3637053e-01,
        -3.5735342e-01,  1.4720384e-02,  5.6321281e-01,  1.0573757e+00,
        -1.0641952e-01, -7.5060594e-01,  1.1950602e-01,  5.7973540e-01,
         7.0914221e-01, -5.5394304e-01,  1.0270479e+00, -8.7821597e-01,
         2.4422202e-01,  6.0519435e-02,  6.4671451e-01,  7.5697160e-01,
        -8.9677095e-01,  1.1938182e-01,  1.0028274e+00,  2.0390332e-01,
         6.8945684e-02, -6.1266530e-01,  2.9391396e-01, -3.91996

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [ ]:
from sklearn.decomposition import PCA


pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(examples['features'])

In [ ]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

In [ ]:
%matplotlib inline

In [ ]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1003', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [ ]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [19]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {e + 1}. Train Loss: {0}")
    model.train()
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        preds = model(features)
        loss = criterion(preds, targets)
        loss.backward()
        optimizer.step()
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            preds = model(features)
            loss = criterion(preds, targets)
            acc = (preds.argmax(dim = 1) == targets).float().mean()
            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [ ]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 3e-4
num_epochs = 5

model = nn.Sequential(
    nn.Linear(vector_size,128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32,8),
    nn.ReLU(),
    nn.Linear(8,2),
    nn.Sigmoid()
)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [ ]:
from torchsummary import summary

In [ ]:
summary(model, input_size=(1,300))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 128]          38,528
              ReLU-2               [-1, 1, 128]               0
            Linear-3                [-1, 1, 32]           4,128
              ReLU-4                [-1, 1, 32]               0
            Linear-5                 [-1, 1, 8]             264
              ReLU-6                 [-1, 1, 8]               0
            Linear-7                 [-1, 1, 2]              18
           Sigmoid-8                 [-1, 1, 2]               0
Total params: 42,938
Trainable params: 42,938
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.16
Estimated Total Size (MB): 0.17
----------------------------------------------------------------


Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [ ]:
best_metric = -np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Acc"] > best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Acc"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5839529891014099, 'Test Acc': 0.68891015625}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.641555037021637, 'Test Acc': 0.66937109375}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.6386495280265808, 'Test Acc': 0.67215625}


In [ ]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

# это получается без тфидф и без скользящего окна ниже построим датасеты, собирающие векторы с учетом всего этого
# получилось 70% на первой эпохе, а дальше наблюдалось переобучение. Почему-то аутпуты закрылись

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5853433009153738, 'Test Acc': 0.6875249600638977}


## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

In [17]:
from torch.utils.data import Dataset, random_split


class DatasetWithContext(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens

    def get_embeddings_(self, tokens, window=3):
        embeddings = []
        for i in range(len(tokens)):
            vector = np.zeros(word2vec.vector_size)
            count = 0
            for j in range(1, window + 1):
                if i - j >= 0:
                    if tokens[i-j] in word2vec:
                        vector += (word2vec.get_vector(tokens[i-j]) - self.mean) / self.std
                        count += 1
                if i + j < len(tokens):
                    if tokens[i+j] in word2vec:
                        vector += (word2vec.get_vector(tokens[i+j]) - self.mean) / self.std
                        count += 1
            if count != 0:
                embeddings.append(vector / count) # помимо суммирования добавим и деление на количество векторов, чтобы норма вектора у слов посередине 
            else:                                 # не была всегда выше нормы крайних векторов
                embeddings.append(vector)


        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

Для сохранения выводов и большего удобства скопируем все нужные клетки и запустим

In [18]:
dev_con = DatasetWithContext(dev_data, "text", "emotion", word2vec)

In [19]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev_con) * 0.8)

train, valid = random_split(dev_con, [train_size, len(dev_con) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [20]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev_con.word2vec.vector_size
num_classes = 2
lr = 3e-4
num_epochs = 5

model = nn.Sequential(
    nn.Linear(vector_size ,128),
    nn.ReLU(),
    nn.Linear(128, 32),
    nn.ReLU(),
    nn.Linear(32,8),
    nn.ReLU(),
    nn.Linear(8,2),
    nn.Sigmoid()
)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [21]:
best_metric = -np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Acc"] > best_metric:
        torch.save(model.state_dict(), "model_con.pt")
        best_metric = log["Test Acc"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5913856818675994, 'Test Acc': 0.6898125}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.6429676094055176, 'Test Acc': 0.64465625}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.6484660172462463, 'Test Acc': 0.53085546875}


In [23]:
# как видим, не всегда меньший лосс соответствует лучшей метрике точности, поэтому я решил сохранять лучшую модель относительно точности
test_loader = DataLoader(
    DatasetWithContext(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_con.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.6181059672047917, 'Test Acc': 0.6964825029952076}


Проделаем то же самое для второго задания с TF-IDF

т.к. TF-IDF не проходит по памяти попробуем заодно другую модель векторизации слов

In [11]:
word2vec = api.load("glove-twitter-200")

[==================================================] 100.0% 758.5/758.5MB downloaded


In [12]:
# проделываем всю необходимую подготовку для тф идф а именно строим вектора размерности такой же как и вектора word2vec и собираем в единый словарь, для быстрого доступа
from torch.utils.data import Dataset, random_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sparsesvd import sparsesvd
from scipy import sparse



vectorizer = TfidfVectorizer(stop_words='english', min_df=7, norm=None)
sparse_features = vectorizer.fit_transform(dev_data['text'].values)
ut, s, vt = sparsesvd(sparse.csc_matrix(sparse_features), 200)
vtt = vt.transpose()
print(vtt.shape)
tfidf_dict = {}
for i, word in enumerate(vectorizer.get_feature_names()):
    tfidf_dict[word] = vtt[i]


(49561, 200)


In [14]:
import pickle
a_file = open("tfidf.pkl", "wb")
pickle.dump(tfidf_dict, a_file)
a_file.close()

In [ ]:
a_file = open("tfidf.pkl", "rb")
tfidf_dict = pickle.load(a_file)

In [10]:
import gc
del vectorizer
del sparse_features
del vtt
gc.collect()

0

In [21]:
class DatasetTFIDF(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        
        self.data = data

        self.feature_column = feature_column
        self.target_column = target_column
        self.word2vec = word2vec
         
        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens, text)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        line = self.tokenizer.tokenize(text.lower())
        tokens = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
        return tokens

    def get_embeddings_(self, tokens, text):
        embeddings = []
        for token in tokens:
            vector = np.zeros(word2vec.vector_size)
            if token in word2vec:
                vector += (word2vec.get_vector(token) - self.mean) / self.std
            if token in tfidf_dict.keys():
                vector += tfidf_dict[token]
            embeddings.append(vector)


        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)
        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [22]:
dev_tfidf = DatasetTFIDF(dev_data, "text", "emotion", word2vec)

In [23]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 2

def average_emb(batch):
    features = [np.mean(b["feature"], axis=0) for b in batch]
    targets = [b["target"] for b in batch]

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev_tfidf) * 0.8)

train, valid = random_split(dev_tfidf, [train_size, len(dev_tfidf) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

In [26]:
import torch.nn as nn
from torch.optim import Adam


# Не забудь поиграться с параметрами ;)
vector_size = dev_tfidf.word2vec.vector_size
num_classes = 2
lr = 3e-4
num_epochs = 5

model = nn.Sequential(
    nn.Linear(vector_size ,256),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.ReLU(),
    nn.Linear(128,32),
    nn.ReLU(),
    nn.Linear(32,2),
    nn.Sigmoid()
)
model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [27]:
best_metric = -np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Acc"] > best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Acc"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fabc2943d40>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7fabc2943d40>
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.6252626178264618, 'Test Acc': 0.6863671875}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.7530300736427307, 'Test Acc': 0.55980859375}


Epoch 5. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.770907274723053, 'Test Acc': 0.541921875}


In [29]:
test_loader = DataLoader(
    DatasetTFIDF(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))


Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.6253835825493541, 'Test Acc': 0.6862613568290735}


In [ ]:
#как видим в этом случае наблюдается более выраженное переобучение, однако помним, что здесь датасет меньше и совсем другой